## Assignment: Segmentation and clustering of neighborhood in Toronto

## 1. Fetch the data from wikipedia by scraping 

In [1]:
!pip install folium

     |████████████████████████████████| 93 kB 4.0 MB/s  eta 0:00:01


In [2]:
# First we will import essential libraries
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import folium

In [3]:
main_source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(main_source, 'lxml')

Table = soup.find("table")
Table_row = Table.tbody.find_all('tr')

res = []
for tr in Table_row:
    td = tr.find_all("td")
    row = [tr.text for tr in td]

    if row != [] and row != "Not assigned":
        
        if "Not assigned" in row[2]:
            row[2] = row[0]
        res.append(row)

web_df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
web_df = web_df.replace('\n','',regex=True)
web_df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,M1A
1,M2A,Not assigned,M2A
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,M5Z
176,M6Z,Not assigned,M6Z
177,M7Z,Not assigned,M7Z
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [4]:
web_df = web_df[web_df.Borough != 'Not assigned']
web_df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
web_df1 = web_df.reset_index(drop = True)
web_df1

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
print("shape: ", web_df1.shape)

shape:  (103, 3)


***

## 2. Since over data frame is ready we will merge it with latitude and longitude columns, which we will fetch from geospatial csv file

In [7]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**we have loaded the data, so now we will merge the coordinates data with the data consisting the Borough adn Neighborhood**

In [8]:
toronto_df = pd.merge(web_df1, df_data_1, how='left', left_on = 'PostalCode',
                     right_on = 'Postal Code')
toronto_df.drop("Postal Code", axis=1, inplace=True)
toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


**We have the data frame with the location of the neighborhood and we can now perform clustering with the above data frame**

***

## 3. Exploreing and clustering the toronto neighborhoods.

### Generating coordinates of Toronto city

In [9]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent = 'toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Coordinates for Totonto city {}, {}".format(latitude,longitude))

Coordinates for Totonto city 43.6534817, -79.3839347


### Generate the map of Toronto city with neighborhoods on top:

In [10]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start = 15)
for Lati, Longi, borough, neigh_hood, in zip(toronto_df['Latitude'],toronto_df['Longitude'],toronto_df['Borough'],toronto_df['Neighborhood']):
    label = '{},{}'.format(neigh_hood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([Lati,Longi], radius = 5, popup=label,
                       color = 'blue',fill=True, fill_color = '#3186cc',
                       fill_opacity = 0.5, parse_html=False).add_to(toronto_map)

toronto_map 

**We have generated the map and points on Toronto city in the above visualization**

### Now, we will work with the borough having name "Toronto" OR contain the word "Toronto"

In [11]:
toronto_df_boro = toronto_df[toronto_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_df_boro

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


**Plotting the borough having Toronto in their name, and placing the markers in that region**

In [12]:
toronto_boro_map = folium.Map(location=[latitude,longitude], zoom_start = 15)
for Lati, Longi, borough, neigh_hood, in zip(toronto_df_boro['Latitude'],toronto_df_boro['Longitude'],toronto_df_boro['Borough'],toronto_df_boro['Neighborhood']):
    label = '{},{}'.format(neigh_hood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([Lati,Longi], radius = 5, popup=label,
                       color = 'blue',fill=True, fill_color = '#3186cc',
                       fill_opacity = 0.5, parse_html=False).add_to(toronto_boro_map)
toronto_boro_map

***

## Define Foursquare credentials and Version:

In [13]:
CLIENT_ID = 'HS3X0XNID2Q1QMHZS500CGYGK1WVOD5RMZ2KGSMDH0LA41RP' # your Foursquare ID
CLIENT_SECRET = 'PKT3NGYANXY0SSWYTCEGV4GZ54LUQTVUUR0RWVAKEYTINKY1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HS3X0XNID2Q1QMHZS500CGYGK1WVOD5RMZ2KGSMDH0LA41RP
CLIENT_SECRET:PKT3NGYANXY0SSWYTCEGV4GZ54LUQTVUUR0RWVAKEYTINKY1


**Now we will explore the neighborhood in datafreame**

In [14]:
# Lets start by exploring the first Neighborhood in over dataframe.

toronto_df_boro.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [15]:
neighborhood_latitude = toronto_df_boro.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df_boro.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df_boro.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


#### Now, we will get the top 100 values within 500 meter radious in Regent Park, Harbourfront area.

In [16]:
# Here we are creating a get request using the url.
LIMIT = 100
radious = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radious={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radious, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=HS3X0XNID2Q1QMHZS500CGYGK1WVOD5RMZ2KGSMDH0LA41RP&client_secret=PKT3NGYANXY0SSWYTCEGV4GZ54LUQTVUUR0RWVAKEYTINKY1&v=20180605&ll=43.6542599,-79.3606359&radious=500&limit=100'

In [17]:
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5fde8b886ef73c60c615197a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 1150,
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 138,
  'suggestedBounds': {'ne': {'lat': 43.66423449087944,
    'lng': -79.34745925969683},
   'sw': {'lat': 43.64627169153188, 'lng': -79.37572993584668}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        

In [18]:
# now we will generate the get category function to get the information from the items
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now we will change the JSON data into Pandas Data frame

In [19]:
venues = result['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Impact Kitchen,Restaurant,43.656369,-79.356980
3,The Distillery Historic District,Historic Site,43.650244,-79.359323
4,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008


In [20]:
# Now lets check the values returned by foresquare
print("{} venues returned by foursquare".format(nearby_venues.shape[0]))

100 venues returned by foursquare


***

## 4. Explore neighborhood in Toronto

In [29]:
# We will create a function that will repeat the above operation for all the neighborhoods in Toronto
def GetNearbyVenue(names, latitudes, longitudes, radius=500):
    
    venue_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # creating Url, API request
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radious={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radious, LIMIT)
        
        result = requests.get(url).json()["response"]["groups"][0]['items']
        
        venue_list.append([(name, lat,lng,v['venue']['name'],
                                             v['venue']['location']['lat'],
                                             v['venue']['location']['lng'],
                                             v['venue']['categories'][0]['name'])for v in result])
    
    nearby_venues = pd.DataFrame([items for venue_list in venue_list for items in venue_list])
    nearby_venues.columns = ['Neighborhood','Neighborhood Latitude',
                             'Neighborhood Longitude', 'Venue',
                             'Venue Latitude', 'Venue Longitude',
                             'Venue Category']
    return(nearby_venues)

In [30]:
# New data frame for totonto neighborhood
toronto_venue = GetNearbyVenue(names=toronto_df_boro['Neighborhood'],
                               latitudes = toronto_df_boro['Latitude'],
                               longitudes = toronto_df_boro['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


**Let us check the size of the dataframe**

In [31]:
print(toronto_venue.shape)
toronto_venue.head()

(3847, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
3,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site
4,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


In [33]:
# Let us check the venues returned by each neighbors
toronto_venue.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",100,100,100,100,100,100
Central Bay Street,86,86,86,86,86,86
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100


### Unique categories returned by venues

In [35]:
print('There are {} unique categories.'.format(len(toronto_venue['Venue Category'].unique())))

There are 284 unique categories.


***

## Analyze each neighborhood

In [36]:
# we will use one hot encoding to add neighborhood column back to data frame and 
# shift it to first column

toronto_encoding = pd.get_dummies(toronto_venue[['Venue Category']], prefix="", prefix_sep="")

toronto_encoding['Neighborhood'] = toronto_venue['Neighborhood']

fix_column = [toronto_encoding.columns[-1]] + list(toronto_encoding.columns[:-1])
toronto_encoding = toronto_encoding[fix_column]
toronto_encoding.head()

,Zoo,Adult Boutique,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,...,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
toronto_encoding.shape

(3847, 284)

**Grouping by neighborhood and getting the frequency of the occurence of each neighborhood**

In [39]:
toronto_group = toronto_encoding.groupby('Neighborhood').mean().reset_index()
toronto_group

,Neighborhood,Zoo,Adult Boutique,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.01,0.000000,...,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.010000
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.000000,0.00,0.00,0.02,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.010000
2,"Business reply mail Processing Centre, South C...",0.00,0.000000,0.00,0.00,0.02,0.00,0.00,0.01,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.00,0.00,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.000000,0.01,0.01,0.00,0.00,0.00,0.00,0.030000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.020000
4,Central Bay Street,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.011628,0.011628,0.011628,0.000000,0.011628,0.00,0.00,0.00,0.011628
5,Christie,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.01,0.00,0.010000
6,Church and Wellesley,0.00,0.010000,0.00,0.00,0.01,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.00,0.00,0.020000
7,"Commerce Court, Victoria Hotel",0.00,0.000000,0.00,0.00,0.03,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.020000,0.000000,0.000000,0.000000,0.00,0.01,0.00,0.000000
8,Davisville,0.00,0.000000,0.00,0.00,0.01,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.010000,0.000000,0.000000,0.010000,0.00,0.01,0.02,0.020000
9,Davisville North,0.00,0.000000,0.00,0.00,0.01,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.010000,0.000000,0.000000,0.010000,0.00,0.01,0.01,0.020000


In [40]:
toronto_group.shape

(39, 284)

## 5. Display each neighborhood with top 5 most common values

In [42]:
Top_venue = 5

for hood in toronto_group['Neighborhood']:
    print("------"+hood+"-------")
    temp = toronto_group[toronto_group['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop=True).head(Top_venue))
    print('\n')

------Berczy Park-------
                 venue  freq
0          Coffee Shop  0.10
1                Hotel  0.05
2                 Café  0.04
3             Beer Bar  0.04
4  Japanese Restaurant  0.04


------Brockton, Parkdale Village, Exhibition Place-------
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2       Bakery  0.05
3          Bar  0.04
4   Restaurant  0.04


------Business reply mail Processing Centre, South Central Letter Processing Plant Toronto-------
         venue  freq
0         Park  0.07
1  Coffee Shop  0.06
2      Brewery  0.05
3        Beach  0.04
4         Café  0.04


------CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport-------
         venue  freq
0         Park  0.10
1  Coffee Shop  0.05
2         Café  0.05
3        Hotel  0.03
4          Gym  0.03


------Central Bay Street-------
             venue  freq
0      Coffee Shop  0.14
1             Café  0.08
2   Sandwich Place  0.03
3      

***

#### Now we will put the generated result into pandas data frame:

In [48]:
# First we will wright the function to sort the venues in descending order.

def common_venue(row, Top_venue):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:Top_venue]

In [49]:
Top_venue = 10

indicators = ['st', 'nd', 'rd']

columns = ["Neighborhood"]
for indx in np.arange(Top_venue):
    try:
        columns.append('{}{} Most Common Venue'.format(indx+1, indicators[indx]))
    except:
        columns.append('{}th Most Common Venue'.format(indx+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_group['Neighborhood']

for indx in np.arange(toronto_group.shape[0]):
    neighborhoods_venues_sorted.iloc[indx, 1:] = common_venue(toronto_group.iloc[indx, :], Top_venue)

neighborhoods_venues_sorted.head()    

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Café,Restaurant,Beer Bar,Japanese Restaurant,Cocktail Bar,Park,Italian Restaurant,Bakery
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Bakery,Café,Bar,Restaurant,Furniture / Home Store,Gift Shop,New American Restaurant,American Restaurant,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Park,Coffee Shop,Brewery,Café,Beach,Italian Restaurant,Indian Restaurant,French Restaurant,Bar,Bakery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Park,Café,Coffee Shop,Italian Restaurant,Gym,Hotel,Aquarium,Restaurant,Spa,Music Venue
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Art Gallery,Gastropub,French Restaurant,Bar,Middle Eastern Restaurant,Pizza Place,Thai Restaurant


***

## 6. Cluster Neighborhoods

#### Executing K-Means clustering to cluster neighborhood into 5 clusters.

In [51]:
kclusters = 5

toronto_group_clust = toronto_group.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(toronto_group_clust)
kmeans.labels_[0:10]

array([2, 1, 4, 4, 1, 0, 1, 2, 3, 3], dtype=int32)

In [54]:
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

toronto_merge = toronto_df_boro

toronto_merge = toronto_merge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,1,Coffee Shop,Café,Restaurant,Bakery,Theater,Farmers Market,Park,Pub,Hotel,Thai Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,1,Coffee Shop,Sandwich Place,Café,Park,Sushi Restaurant,Italian Restaurant,Pharmacy,Japanese Restaurant,Gastropub,Yoga Studio
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,1,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Hotel,Bubble Tea Shop,Electronics Store,Restaurant,Sushi Restaurant,Lingerie Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,1,Coffee Shop,Café,Cosmetics Shop,Seafood Restaurant,Gastropub,Bakery,Japanese Restaurant,Theater,American Restaurant,Italian Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,4,Park,Coffee Shop,Beach,Pub,Breakfast Spot,Bakery,Ice Cream Shop,Café,Indian Restaurant,BBQ Joint


***

**Lets visualize the resulting Clusters**

In [57]:
Cluster_mapping = folium.Map(location = [latitude, longitude], zoom_start = 11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merge['Latitude'], toronto_merge['Longitude'], toronto_merge['Neighborhood'], toronto_merge['Cluster Label']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html = True)
    folium.CircleMarker([lat,lon], radius=5, popup=label, color=rainbow[cluster-1], 
                        fill = True, fill_color=rainbow[cluster-1], 
                        fill_opacity=0.7).add_to(Cluster_mapping)
Cluster_mapping

***

## Examining clusters:

**Now, we can examine rach cluster and determine venue categories that distinguish each cluster.**

#### Cluster 1

In [58]:
toronto_merge.loc[toronto_merge['Cluster Label'] == 0, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,0,Café,Coffee Shop,Italian Restaurant,Bakery,Bar,Ice Cream Shop,Indian Restaurant,Grocery Store,South American Restaurant,Comedy Club
11,West Toronto,0,0,Café,Restaurant,Bar,Vegetarian / Vegan Restaurant,Italian Restaurant,Bakery,Cocktail Bar,Coffee Shop,Pizza Place,Asian Restaurant
24,Central Toronto,0,0,Italian Restaurant,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Restaurant,Museum,French Restaurant,Grocery Store,Bakery,Park
27,Downtown Toronto,0,0,Café,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Restaurant,Bookstore,Italian Restaurant,Cheese Shop,Comfort Food Restaurant,Pub
30,Downtown Toronto,0,0,Café,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Caribbean Restaurant,Comfort Food Restaurant,Park,Dessert Shop,Pizza Place


***

#### Cluster 2

In [59]:
toronto_merge.loc[toronto_merge['Cluster Label'] == 1, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,1,Coffee Shop,Café,Restaurant,Bakery,Theater,Farmers Market,Park,Pub,Hotel,Thai Restaurant
1,Downtown Toronto,1,1,Coffee Shop,Sandwich Place,Café,Park,Sushi Restaurant,Italian Restaurant,Pharmacy,Japanese Restaurant,Gastropub,Yoga Studio
2,Downtown Toronto,1,1,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Hotel,Bubble Tea Shop,Electronics Store,Restaurant,Sushi Restaurant,Lingerie Store
3,Downtown Toronto,1,1,Coffee Shop,Café,Cosmetics Shop,Seafood Restaurant,Gastropub,Bakery,Japanese Restaurant,Theater,American Restaurant,Italian Restaurant
6,Downtown Toronto,1,1,Coffee Shop,Café,Sandwich Place,Art Gallery,Gastropub,French Restaurant,Bar,Middle Eastern Restaurant,Pizza Place,Thai Restaurant
14,West Toronto,1,1,Coffee Shop,Bakery,Café,Bar,Restaurant,Furniture / Home Store,Gift Shop,New American Restaurant,American Restaurant,Italian Restaurant
35,Downtown Toronto,1,1,Coffee Shop,Café,Japanese Restaurant,Park,Pub,Restaurant,Gastropub,Diner,Thai Restaurant,Men's Store
37,Downtown Toronto,1,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Café,Mediterranean Restaurant,Juice Bar,Diner,Thai Restaurant


***

#### Cluster 3

In [60]:
toronto_merge.loc[toronto_merge['Cluster Label'] == 2, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,2,2,Coffee Shop,Hotel,Café,Restaurant,Beer Bar,Japanese Restaurant,Cocktail Bar,Park,Italian Restaurant,Bakery
8,Downtown Toronto,2,2,Café,Coffee Shop,Restaurant,Hotel,Sushi Restaurant,Thai Restaurant,Theater,Gym,Arts & Crafts Store,Steakhouse
10,Downtown Toronto,2,2,Coffee Shop,Hotel,Boat or Ferry,Scenic Lookout,Brewery,Park,Ice Cream Shop,Japanese Restaurant,Pizza Place,Aquarium
13,Downtown Toronto,2,2,Coffee Shop,Hotel,Café,Restaurant,Gym,Japanese Restaurant,Seafood Restaurant,American Restaurant,Concert Hall,Italian Restaurant
16,Downtown Toronto,2,2,Coffee Shop,Restaurant,Hotel,Café,Gym,Japanese Restaurant,Seafood Restaurant,American Restaurant,Italian Restaurant,Burger Joint
34,Downtown Toronto,2,2,Coffee Shop,Café,Restaurant,Japanese Restaurant,Beer Bar,Seafood Restaurant,Hotel,Italian Restaurant,Gym,Bakery
36,Downtown Toronto,2,2,Hotel,Coffee Shop,Café,Restaurant,Japanese Restaurant,Asian Restaurant,American Restaurant,Seafood Restaurant,Gym,Steakhouse


***

#### Cluster 4

In [61]:
toronto_merge.loc[toronto_merge['Cluster Label'] == 3, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,West Toronto,3,3,Café,Italian Restaurant,Coffee Shop,Bar,Bakery,Park,Breakfast Spot,Cocktail Bar,Brewery,Thai Restaurant
18,Central Toronto,3,3,Coffee Shop,Italian Restaurant,Park,Café,Bakery,Sushi Restaurant,Yoga Studio,Spa,Pizza Place,Supermarket
19,Central Toronto,3,3,Italian Restaurant,Coffee Shop,Café,Restaurant,Bakery,Middle Eastern Restaurant,Japanese Restaurant,Sushi Restaurant,Food & Drink Shop,Mexican Restaurant
20,Central Toronto,3,3,Coffee Shop,Italian Restaurant,Café,Bakery,Indian Restaurant,Restaurant,Pizza Place,Park,Breakfast Spot,Deli / Bodega
21,Central Toronto,3,3,Coffee Shop,Italian Restaurant,Sushi Restaurant,Japanese Restaurant,Café,Bank,Gym,Restaurant,Middle Eastern Restaurant,Park
22,West Toronto,3,3,Coffee Shop,Café,Brewery,Italian Restaurant,Bakery,Bar,Gastropub,Pizza Place,Restaurant,Liquor Store
23,Central Toronto,3,3,Italian Restaurant,Coffee Shop,Café,Sushi Restaurant,Bakery,Restaurant,Park,Pizza Place,Bookstore,Japanese Restaurant
25,West Toronto,3,3,Coffee Shop,Café,Bakery,Park,Bar,Italian Restaurant,Restaurant,Gastropub,Breakfast Spot,Dog Run
26,Central Toronto,3,3,Italian Restaurant,Coffee Shop,Café,Pizza Place,Bakery,Restaurant,Indian Restaurant,Park,Japanese Restaurant,Yoga Studio
28,West Toronto,3,3,Coffee Shop,Italian Restaurant,Bakery,Park,Bar,Café,Scenic Lookout,Gastropub,Brewery,Pizza Place


***

#### Cluster 5

In [62]:
toronto_merge.loc[toronto_merge['Cluster Label'] == 4, toronto_merge.columns[[1] + list(range(5, toronto_merge.shape[1]))]]

,Borough,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,4,4,Park,Coffee Shop,Beach,Pub,Breakfast Spot,Bakery,Ice Cream Shop,Café,Indian Restaurant,BBQ Joint
12,East Toronto,4,4,Greek Restaurant,Park,Café,Bakery,Vietnamese Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Restaurant,Brewery
15,East Toronto,4,4,Coffee Shop,Park,Beach,Brewery,Café,BBQ Joint,Bar,American Restaurant,Italian Restaurant,Indian Restaurant
17,East Toronto,4,4,Coffee Shop,Park,Vietnamese Restaurant,Brewery,Café,Bakery,French Restaurant,Diner,Bar,Italian Restaurant
32,Downtown Toronto,4,4,Park,Café,Coffee Shop,Italian Restaurant,Gym,Hotel,Aquarium,Restaurant,Spa,Music Venue
38,East Toronto,4,4,Park,Coffee Shop,Brewery,Café,Beach,Italian Restaurant,Indian Restaurant,French Restaurant,Bar,Bakery


***